In [1]:
import os
import urllib.request

In [2]:
from gromacs_py import gmx
gmx.show_log()

In [3]:
from gromacs_py.tools import ambertools
ambertools.show_log()

In [4]:
cd /home/murail/Documents/Code/gromacs_py/docs/notebook/

/home/murail/Documents/Code/gromacs_py/docs/notebook


In [5]:
rm -r data_lig

In [6]:
DATA_OUT = 'data_lig'
PDB_ID = '1LI2'
os.makedirs(DATA_OUT, exist_ok = True)

raw_pdb = urllib.request.urlretrieve('http://files.rcsb.org/download/{}.pdb'.format(PDB_ID),
                           '{}/{}.pdb'.format(DATA_OUT, PDB_ID))

In [7]:
md_sys = gmx.GmxSys(name=PDB_ID, coor_file=raw_pdb[0])
md_sys.prepare_top(out_folder=os.path.join(DATA_OUT, 'prot_top'),
                   ff='amber99sb-ildn', include_mol=['IPH', 'BME'])

Succeed to read file ../1LI2.pdb ,  1373 atoms found
Succeed to read file ../1LI2.pdb ,  1373 atoms found
Succeed to save file tmp_pdb2pqr.pdb
pdb2pqr.py --ff AMBER --ffout AMBER --chain --ph-calc-method=propka tmp_pdb2pqr.pdb 00_1LI2.pqr
Succeed to read file 00_1LI2.pqr ,  2601 atoms found
residue(s) HOH not included, add this residue in ...
Succeed to read file ../1LI2.pdb ,  1373 atoms found
Succeed to save file BME.pdb
reduce -build -nuclear BME.pdb
Succeed to save file BME_h.pdb
Succeed to read file BME_h.pdb ,  20 atoms found
Succeed to save file BME_h_unique.pdb
acpype -i BME_h_unique.pdb -b BME -c bcc -a gaff -o gmx
Succeed to create topologie in BME
BME

residue(s) CL not included, add this residue in ...
Succeed to read file ../1LI2.pdb ,  1373 atoms found
Succeed to save file IPH.pdb
reduce -build -nuclear IPH.pdb
Succeed to save file IPH_h.pdb
Succeed to read file IPH_h.pdb ,  13 atoms found
Succeed to save file IPH_h_unique.pdb
acpype -i IPH_h_unique.pdb -b IPH -c bcc -a g

In [8]:
md_sys.em_2_steps(out_folder=os.path.join(DATA_OUT, 'prot_em'),
                  no_constr_nsteps=10000,
                  constr_nsteps=10000,
                  posres="",
                  emtol=0.1, nstxout=100, maxwarn=3,
                  create_box_flag=True)

-Create pbc box
gmx editconf -f tmp_2/prot_top/1LI2_pdb2gmx_mol.pdb -o tmp_2/prot_top/1LI2_pdb2gmx_mol_box.pdb -bt dodecahedron -d 1.0
WARNING !!! ADDING unusual parameter : "emtol"in the mdp file None
-Create the tpr file Init_em_1LI2.tpr
gmx grompp -f Init_em_1LI2.mdp -c ../prot_top/1LI2_pdb2gmx_mol_box.pdb -r ../prot_top/1LI2_pdb2gmx_mol_box.pdb -p ../prot_top/1LI2_pdb2gmx_mol.top -po out_Init_em_1LI2.mdp -o Init_em_1LI2.tpr -maxwarn 3
-Launch the simulation Init_em_1LI2.tpr
gmx mdrun -s Init_em_1LI2.tpr -deffnm Init_em_1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright



WARNING !!! ADDING unusual parameter : "emtol"in the mdp file Init_em_1LI2.mdp
-Create the tpr file 1LI2.tpr
gmx grompp -f 1LI2.mdp -c Init_em_1LI2.gro -r Init_em_1LI2.gro -p ../prot_top/1LI2_pdb2gmx_mol.top -po out_1LI2.mdp -o 1LI2.tpr -maxwarn 3
-Launch the simulation 1LI2.tpr
gmx mdrun -s 1LI2.tpr -deffnm 1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [9]:
md_sys.solvate_add_ions(out_folder=os.path.join(DATA_OUT, 'sys_top'),
                        ion_C=0.15, maxwarn=3, create_box_flag=False)
md_sys.display()


-Solvate the pbc box
Copy topologie file and dependancies

Copy topologie file and dependancies

-Create the tpr file genion_1LI2_water_ion.tpr
gmx grompp -f ../../../../gromacs_py/template/mini.mdp -c 1LI2_water.pdb -r 1LI2_water.pdb -p 1LI2_water_ion.top -po out_mini.mdp -o genion_1LI2_water_ion.tpr -maxwarn 3
-Add ions to the system with an ionic concentration of 0.15 M , sytem charge = 8.0 water num= 10118
Add ions : NA : 27   CL : 35
gmx genion -s genion_1LI2_water_ion.tpr -p 1LI2_water_ion.top -o 1LI2_water_ion.gro -np 27 -pname NA -nn 35 -nname CL
name         : 1LI2
sim_name     : genion_1LI2_water_ion
coor_file    : tmp_2/sys_top/1LI2_water_ion.gro
top_file     : tmp_2/sys_top/1LI2_water_ion.top
tpr          : tmp_2/sys_top/genion_1LI2_water_ion.tpr
mdp          : ../../gromacs_py/template/mini.mdp
xtc          : tmp_2/prot_em/1LI2.trr
edr          : tmp_2/prot_em/1LI2.edr
log          : tmp_2/prot_em/1LI2.log
nt           : 0
ntmpi        : 0
sys_history  : 2


In [10]:
md_sys.em_2_steps(out_folder=os.path.join(DATA_OUT, 'sys_em'),
                  no_constr_nsteps=10000,
                  constr_nsteps=10000,
                  posres="",maxwarn=3)

-Create the tpr file Init_em_1LI2.tpr
gmx grompp -f Init_em_1LI2.mdp -c ../sys_top/1LI2_water_ion.gro -r ../sys_top/1LI2_water_ion.gro -p ../sys_top/1LI2_water_ion.top -po out_Init_em_1LI2.mdp -o Init_em_1LI2.tpr -maxwarn 3
-Launch the simulation Init_em_1LI2.tpr
gmx mdrun -s Init_em_1LI2.tpr -deffnm Init_em_1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright



-Create the tpr file 1LI2.tpr
gmx grompp -f 1LI2.mdp -c Init_em_1LI2.gro -r Init_em_1LI2.gro -p ../sys_top/1LI2_water_ion.top -po out_1LI2.mdp -o 1LI2.tpr -maxwarn 3
-Launch the simulation 1LI2.tpr
gmx mdrun -s 1LI2.tpr -deffnm 1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [11]:
mdp_options = {'nsteps': 20000,
               'define': '-DPOSRES', 'dt': 0.0005}
equi_template_mdp = os.path.join(gmx.GROMACS_MOD_DIRNAME,
                                 "template/equi.mdp")
md_sys.run_md_sim(out_folder=os.path.join(DATA_OUT, "sys_short_equi"),
                  name="equi_HA_short_" + PDB_ID,
                  mdp_template=equi_template_mdp,
                  mdp_options=mdp_options, maxwarn=3)

-Create the tpr file equi_HA_short_1LI2.tpr
gmx grompp -f equi_HA_short_1LI2.mdp -c ../sys_em/1LI2.gro -r ../sys_em/1LI2.gro -p ../sys_top/1LI2_water_ion.top -po out_equi_HA_short_1LI2.mdp -o equi_HA_short_1LI2.tpr -maxwarn 3
-Launch the simulation equi_HA_short_1LI2.tpr
gmx mdrun -s equi_HA_short_1LI2.tpr -deffnm equi_HA_short_1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [12]:
HA_time = 0.25
CA_time = 0.5
CA_LOW_time = 1.0
dt = 0.002
dt_HA = 0.001

md_sys.equi_three_step(out_folder=os.path.join(DATA_OUT, 'sys_equi'),
                                     nsteps_HA=1000 * HA_time / dt_HA,
                                     nsteps_CA=1000 * CA_time / dt,
                                     nsteps_CA_LOW=1000 * CA_LOW_time / dt,
                                     dt=dt, dt_HA=dt_HA, maxwarn=3)


-Create the tpr file equi_HA_1LI2.tpr
gmx grompp -f equi_HA_1LI2.mdp -c ../../sys_short_equi/equi_HA_short_1LI2.gro -r ../../sys_short_equi/equi_HA_short_1LI2.gro -p ../../sys_top/1LI2_water_ion.top -po out_equi_HA_1LI2.mdp -o equi_HA_1LI2.tpr -maxwarn 3
-Launch the simulation equi_HA_1LI2.tpr
gmx mdrun -s equi_HA_1LI2.tpr -deffnm equi_HA_1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright



-Create the tpr file equi_CA_1LI2.tpr
gmx grompp -f equi_CA_1LI2.mdp -c ../00_equi_HA/equi_HA_1LI2.gro -r ../../sys_short_equi/equi_HA_short_1LI2.gro -p ../../sys_top/1LI2_water_ion.top -po out_equi_CA_1LI2.mdp -o equi_CA_1LI2.tpr -maxwarn 3
-Launch the simulation equi_CA_1LI2.tpr
gmx mdrun -s equi_CA_1LI2.tpr -deffnm equi_CA_1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright



-Create the tpr file equi_CA_LOW_1LI2.tpr
gmx grompp -f equi_CA_LOW_1LI2.mdp -c ../01_equi_CA/equi_CA_1LI2.gro -r ../../sys_short_equi/equi_HA_short_1LI2.gro -p ../../sys_top/1LI2_water_ion.top -po out_equi_CA_LOW_1LI2.mdp -o equi_CA_LOW_1LI2.tpr -maxwarn 3
-Launch the simulation equi_CA_LOW_1LI2.tpr
gmx mdrun -s equi_CA_LOW_1LI2.tpr -deffnm equi_CA_LOW_1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [6]:
sys_top = gmx.TopSys('tmp_2/prot_top/IPH.acpype/IPH_GMX.top')
atomtypes_itp = gmx.Itp('IPH_GMX_atomtypes', 'IPH_GMX_atomtypes.itp','tmp_2/prot_top/IPH.acpype/IPH_GMX_atomtypes.itp')

sys_top.itp_list = [atomtypes_itp] + sys_top.itp_list

sys_top.display()

OSError: File cound not be founded :tmp_2/prot_top/IPH.acpype/IPH_GMX.top

In [11]:
test = [1, 2, 3]
print(*test)

1 2 3


In [12]:
sys_top.write_file('tmp_2/prot_top/IPH.acpype/IPH_GMX_test.top')

In [8]:
atomtypes_itp = gmx.Itp('IPH_GMX_atomtypes', 'IPH_GMX_atomtypes.itp','tmp_2/prot_top/IPH.acpype/IPH_GMX_atomtypes.itp')

```
; IPH_GMX.itp created by acpype (v: 2020-07-25T09:06:13CEST) on Wed Sep 16 16:02:33 2020

[ atomtypes ]
;name   bond_type     mass     charge   ptype   sigma         epsilon       Amb
 ca       ca          0.00000  0.00000   A     3.39967e-01   3.59824e-01 ; 1.91  0.0860
 oh       oh          0.00000  0.00000   A     3.06647e-01   8.80314e-01 ; 1.72  0.2104
 ho       ho          0.00000  0.00000   A     0.00000e+00   0.00000e+00 ; 0.00  0.0000
 ha       ha          0.00000  0.00000   A     2.59964e-01   6.27600e-02 ; 1.46  0.0150

[ moleculetype ]
;name            nrexcl
 IPH              3

[ atoms ]
;   nr  type  resi  res  atom  cgnr     charge      mass       ; qtot   bond_type
     1   ca     1   IPH    C1    1     0.123100     12.01000 ; qtot 0.123
     2   ca     1   IPH    C2    2    -0.185000     12.01000 ; qtot -0.062
     3   ca     1   IPH    C3    3    -0.094500     12.01000 ; qtot -0.156
     4   ca     1   IPH    C4    4    -0.166000     12.01000 ; qtot -0.322
     5   ca     1   IPH    C5    5    -0.094500     12.01000 ; qtot -0.417
     6   ca     1   IPH    C6    6    -0.185000     12.01000 ; qtot -0.602
     7   oh     1   IPH    O1    7    -0.498100     16.00000 ; qtot -1.100
     8   ho     1   IPH   HO1    8     0.418000      1.00800 ; qtot -0.682
     9   ha     1   IPH    H6    9     0.141500      1.00800 ; qtot -0.540
    10   ha     1   IPH    H5   10     0.133000      1.00800 ; qtot -0.407
    11   ha     1   IPH    H4   11     0.133000      1.00800 ; qtot -0.274
    12   ha     1   IPH    H3   12     0.133000      1.00800 ; qtot -0.141
    13   ha     1   IPH    H2   13     0.141500      1.00800 ; qtot 0.000

[ bonds ]
;   ai     aj funct   r             k
     1      2   1    1.3984e-01    3.8585e+05 ;     C1 - C2    
     1      6   1    1.3984e-01    3.8585e+05 ;     C1 - C6    
     1      7   1    1.3637e-01    3.2133e+05 ;     C1 - O1    
     2      3   1    1.3984e-01    3.8585e+05 ;     C2 - C3    
     2     13   1    1.0860e-01    2.8937e+05 ;     C2 - H2    
     3      4   1    1.3984e-01    3.8585e+05 ;     C3 - C4    
     3     12   1    1.0860e-01    2.8937e+05 ;     C3 - H3    
     4      5   1    1.3984e-01    3.8585e+05 ;     C4 - C5    
     4     11   1    1.0860e-01    2.8937e+05 ;     C4 - H4    
     5      6   1    1.3984e-01    3.8585e+05 ;     C5 - C6    
     5     10   1    1.0860e-01    2.8937e+05 ;     C5 - H5    
     6      9   1    1.0860e-01    2.8937e+05 ;     C6 - H6    
     7      8   1    9.7300e-02    3.1079e+05 ;     O1 - HO1   

[ pairs ]
;   ai     aj    funct
     1      4      1 ;     C1 - C4    
     1     10      1 ;     C1 - H5    
     1     12      1 ;     C1 - H3    
     2      5      1 ;     C2 - C5    
     2      8      1 ;     C2 - HO1   
     2      9      1 ;     C2 - H6    
     2     11      1 ;     C2 - H4    
     3     10      1 ;     C3 - H5    
     4      9      1 ;     C4 - H6    
     4     13      1 ;     C4 - H2    
     5     12      1 ;     C5 - H3    
     6      3      1 ;     C6 - C3    
     6      8      1 ;     C6 - HO1   
     6     11      1 ;     C6 - H4    
     6     13      1 ;     C6 - H2    
     7      3      1 ;     O1 - C3    
     7      5      1 ;     O1 - C5    
     7      9      1 ;     O1 - H6    
     7     13      1 ;     O1 - H2    
     9     10      1 ;     H6 - H5    
    10     11      1 ;     H5 - H4    
    11     12      1 ;     H4 - H3    
    12     13      1 ;     H3 - H2    

[ angles ]
;   ai     aj     ak    funct   theta         cth
     1      2      3      1    1.2002e+02    5.5731e+02 ;     C1 - C2     - C3    
     1      2     13      1    1.1988e+02    4.0334e+02 ;     C1 - C2     - H2    
     1      6      5      1    1.2002e+02    5.5731e+02 ;     C1 - C6     - C5    
     1      6      9      1    1.1988e+02    4.0334e+02 ;     C1 - C6     - H6    
     1      7      8      1    1.0858e+02    4.1003e+02 ;     C1 - O1     - HO1   
     2      1      6      1    1.2002e+02    5.5731e+02 ;     C2 - C1     - C6    
     2      1      7      1    1.1990e+02    5.8158e+02 ;     C2 - C1     - O1    
     2      3      4      1    1.2002e+02    5.5731e+02 ;     C2 - C3     - C4    
     2      3     12      1    1.1988e+02    4.0334e+02 ;     C2 - C3     - H3    
     3      2     13      1    1.1988e+02    4.0334e+02 ;     C3 - C2     - H2    
     3      4      5      1    1.2002e+02    5.5731e+02 ;     C3 - C4     - C5    
     3      4     11      1    1.1988e+02    4.0334e+02 ;     C3 - C4     - H4    
     4      3     12      1    1.1988e+02    4.0334e+02 ;     C4 - C3     - H3    
     4      5      6      1    1.2002e+02    5.5731e+02 ;     C4 - C5     - C6    
     4      5     10      1    1.1988e+02    4.0334e+02 ;     C4 - C5     - H5    
     5      4     11      1    1.1988e+02    4.0334e+02 ;     C5 - C4     - H4    
     5      6      9      1    1.1988e+02    4.0334e+02 ;     C5 - C6     - H6    
     6      1      7      1    1.1990e+02    5.8158e+02 ;     C6 - C1     - O1    
     6      5     10      1    1.1988e+02    4.0334e+02 ;     C6 - C5     - H5    

[ dihedrals ] ; propers
; for gromacs 4.5 or higher, using funct 9
;    i      j      k      l   func   phase     kd      pn
     1      2      3      4      9   180.00  15.16700   2 ;     C1-    C2-    C3-    C4
     1      2      3     12      9   180.00  15.16700   2 ;     C1-    C2-    C3-    H3
     1      6      5      4      9   180.00  15.16700   2 ;     C1-    C6-    C5-    C4
     1      6      5     10      9   180.00  15.16700   2 ;     C1-    C6-    C5-    H5
     2      1      6      5      9   180.00  15.16700   2 ;     C2-    C1-    C6-    C5
     2      1      6      9      9   180.00  15.16700   2 ;     C2-    C1-    C6-    H6
     2      1      7      8      9   180.00   3.76560   2 ;     C2-    C1-    O1-   HO1
     2      3      4      5      9   180.00  15.16700   2 ;     C2-    C3-    C4-    C5
     2      3      4     11      9   180.00  15.16700   2 ;     C2-    C3-    C4-    H4
     3      4      5      6      9   180.00  15.16700   2 ;     C3-    C4-    C5-    C6
     3      4      5     10      9   180.00  15.16700   2 ;     C3-    C4-    C5-    H5
     4      3      2     13      9   180.00  15.16700   2 ;     C4-    C3-    C2-    H2
     4      5      6      9      9   180.00  15.16700   2 ;     C4-    C5-    C6-    H6
     5      4      3     12      9   180.00  15.16700   2 ;     C5-    C4-    C3-    H3
     6      1      2      3      9   180.00  15.16700   2 ;     C6-    C1-    C2-    C3
     6      1      2     13      9   180.00  15.16700   2 ;     C6-    C1-    C2-    H2
     6      1      7      8      9   180.00   3.76560   2 ;     C6-    C1-    O1-   HO1
     6      5      4     11      9   180.00  15.16700   2 ;     C6-    C5-    C4-    H4
     7      1      2      3      9   180.00  15.16700   2 ;     O1-    C1-    C2-    C3
     7      1      2     13      9   180.00  15.16700   2 ;     O1-    C1-    C2-    H2
     7      1      6      5      9   180.00  15.16700   2 ;     O1-    C1-    C6-    C5
     7      1      6      9      9   180.00  15.16700   2 ;     O1-    C1-    C6-    H6
     9      6      5     10      9   180.00  15.16700   2 ;     H6-    C6-    C5-    H5
    10      5      4     11      9   180.00  15.16700   2 ;     H5-    C5-    C4-    H4
    11      4      3     12      9   180.00  15.16700   2 ;     H4-    C4-    C3-    H3
    12      3      2     13      9   180.00  15.16700   2 ;     H3-    C3-    C2-    H2

[ dihedrals ] ; impropers
; treated as propers in GROMACS to use correct AMBER analytical function
;    i      j      k      l   func   phase     kd      pn
     1      3      2     13      4   180.00   4.60240   2 ;     C1-    C3-    C2-    H2
     1      5      6      9      4   180.00   4.60240   2 ;     C1-    C5-    C6-    H6
     2      4      3     12      4   180.00   4.60240   2 ;     C2-    C4-    C3-    H3
     3      5      4     11      4   180.00   4.60240   2 ;     C3-    C5-    C4-    H4
     4      6      5     10      4   180.00   4.60240   2 ;     C4-    C6-    C5-    H5
     7      1      6      2      4   180.00   4.60240   2 ;     O1-    C1-    C6-    C2
```

In [7]:
md_sys.em(os.path.join(DATA_OUT, 'prot_em'), create_box_flag=True, maxwarn=3)

-Create pbc box
gmx editconf -f tmp_2/prot_top/1LI2_pdb2gmx_mol.pdb -o tmp_2/prot_top/1LI2_pdb2gmx_mol_box.pdb -bt dodecahedron -d 1.0
-Create the tpr file 1LI2.tpr
gmx grompp -f 1LI2.mdp -c ../prot_top/1LI2_pdb2gmx_mol_box.pdb -r ../prot_top/1LI2_pdb2gmx_mol_box.pdb -p ../prot_top/1LI2_pdb2gmx_mol.top -po out_1LI2.mdp -o 1LI2.tpr -maxwarn 3
-Launch the simulation 1LI2.tpr
gmx mdrun -s 1LI2.tpr -deffnm 1LI2 -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [4]:
pdb_in = "phenol.pdb"
pdb_out = "phenol_h.pdb"
mol2_out = "phenol_h.mol2"
dir_out ="tmp"
ambertools.add_hydrogen(pdb_in, pdb_out)
test = ambertools.acpype(pdb_out, dir_out)



PDB files not created, phenol_h.pdb already exist
acpype -i phenol_h.pdb -b tmp -c bcc -a gaff -o gmx
Succeed to create topologie in tmp


In [13]:
view = md_sys.view_coor()
view

_ColormakerRegistry()

-Convert trj/coor
gmx trjconv -f tmp_2/sys_equi/02_equi_CA_LOW/equi_CA_LOW_1LI2.gro -o tmp_2/sys_equi/02_equi_CA_LOW/equi_CA_LOW_1LI2_compact.pdb -s tmp_2/sys_equi/02_equi_CA_LOW/equi_CA_LOW_1LI2.tpr -ur compact -pbc none
Succeed to read file tmp_2/sys_equi/02_equi_CA_LOW/equi_CA_LOW_1LI2_compact.pdb ,  32864 atoms found


NGLWidget()

In [33]:
from pdb_manip_py import pdb_manip

test = pdb_manip.Coor('tmp_2/1LI2.pdb')

res_name_list = []
for atom in test.atom_dict.values():
    res_name_list.append(atom['res_name'])
set(res_name_list)

Succeed to read file tmp_2/1LI2.pdb ,  1373 atoms found


{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'BME',
 'CL',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'HOH',
 'ILE',
 'IPH',
 'LEU',
 'LYS',
 'MET',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'VAL'}

In [35]:
test.get_attribute_selection(attribute='res_name')

['HOH',
 'GLN',
 'IPH',
 'TYR',
 'GLY',
 'LYS',
 'ASP',
 'PRO',
 'VAL',
 'TRP',
 'ILE',
 'ASN',
 'LEU',
 'HIS',
 'THR',
 'ALA',
 'CL',
 'BME',
 'ARG',
 'GLU',
 'SER',
 'CYS',
 'PHE',
 'MET']

In [34]:
test.get_aa_seq()

{'A': 'MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNCNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRCAAINQVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYK'}

In [31]:
test.atom_dict

{0: {'field': 'ATOM',
  'num': 1,
  'name': 'N',
  'alter_loc': ' ',
  'res_name': 'MET',
  'chain': 'A',
  'res_num': 1,
  'uniq_resid': 0,
  'insert_res': ' ',
  'xyz': array([43.991, -3.256,  9.035]),
  'occ': 1.0,
  'beta': 36.36},
 1: {'field': 'ATOM',
  'num': 2,
  'name': 'CA',
  'alter_loc': ' ',
  'res_name': 'MET',
  'chain': 'A',
  'res_num': 1,
  'uniq_resid': 0,
  'insert_res': ' ',
  'xyz': array([43.431, -1.927,  9.119]),
  'occ': 1.0,
  'beta': 18.33},
 2: {'field': 'ATOM',
  'num': 3,
  'name': 'C',
  'alter_loc': ' ',
  'res_name': 'MET',
  'chain': 'A',
  'res_num': 1,
  'uniq_resid': 0,
  'insert_res': ' ',
  'xyz': array([42.027, -1.974,  9.739]),
  'occ': 1.0,
  'beta': 14.57},
 3: {'field': 'ATOM',
  'num': 4,
  'name': 'O',
  'alter_loc': ' ',
  'res_name': 'MET',
  'chain': 'A',
  'res_num': 1,
  'uniq_resid': 0,
  'insert_res': ' ',
  'xyz': array([41.29 , -2.909,  9.525]),
  'occ': 1.0,
  'beta': 20.97},
 4: {'field': 'ATOM',
  'num': 5,
  'name': 'CB',
  'al

In [ ]:
from os_command_py import os_command

os_command.get_directory(dir_out)

In [ ]:
out_folder = 'tmp/tmp1/tmp1/'
out_folder.split('/')

In [ ]:
pdb_in[-3:]

In [ ]:
import os
import logging

from os_command_py import os_command



In [ ]:
REDUCE_BIN = os_command.which('reduce')

reduce_options = {}

# Define reduce command:
cmd_reduce = os_command.Command([REDUCE_BIN,
                                 "-build",
                                 "-nuclear",
                                 pdb_in], **reduce_options)

cmd_reduce.display()
return_code = cmd_reduce.run(display=False, out_data=True)

print(return_code)


In [ ]:
print(return_code['stdout'])